### Some ideas

* walking speed mult
* time mult
* in how many time i want to go a

In [1]:
from collections import defaultdict
import time
import requests
import bs4


class Stop():
    '''stop of each of the routes of GT BUSES'''
    def __init__(self, name, route, _id, _tag, lon, lat):
        self.name = name
        self.route = route
        self._id = _id
        self.tag = _tag
        self.lon = lon
        self.lat = lat
        self.next_buses = []
        
    @property
    def p(self):
        return (self.lon, self.lat)
        
    def __repr__(self):
        return '{}: stop #{} {}, {} {}'.format(self.route, self._id, self.name, self.p, self.next_buses)

In [2]:
def travel_time(s1, s2):
    '''provisionary travel time between 2 stops'''
    return distancepp(s1.p, s2.p)/0.015 *1240 / 3

def app_walking_time(p1, p2):
    '''provisionary walking time between 2 points to save api requests'''
    return distancepp(p1, p2)/0.015 *1240

def stop_waiting_time(s, delay):
    '''provisionary waiting time in a stop'''
    
    
    next_bus = [t for t in s.next_buses if t > float(delay)]
    if len(next_bus) > 0:
        return next_bus[0] - delay
    else:
        return 9999999

def distancepp(p1, p2):
    '''eucladian distance between two points'''
    x1, y1 = p1
    x2, y2 = p2    
    return ((x1-x2)**2 + (y1-y2)**2)**0.5



In [3]:
def init_stops():
    
    '''Open the xml file and gives a dict with the stops of each route'''
    
    with open('routes.xml') as routes_xml:
        path_data = bs4.BeautifulSoup(routes_xml, 'lxml-xml')
    stops = {}
    for k in range(8):
        route_name = path_data.find_all('route')[k].get('tag')
        stops[route_name] = {} 
        for stop in path_data.find_all('route')[k].find_all("stop" ,recursive=False):
            lat = float(stop.get('lat'))
            lon = float(stop.get('lon'))
            stop_name = stop.get('title')
            stop_id = stop.get('stopId')
            tag = stop.get('tag')
            s = Stop(stop_name, route_name, stop_id, tag, lon, lat)
            stops[route_name][tag] = s
            
    return stops

stops = init_stops()


        


### Waiting time to stops:

In [4]:

def add_next_buses_time(stops):
    session = requests.Session()  # Construct a NextBus API compliant requester
    session.headers.update({"User-Agent": "Gonzalo Vargas, CX4242 project gludwig6@gatech.edu"})
    r = session.get('https://gtbuses.herokuapp.com/agencies/georgia-tech/predictions')
    wtdata = bs4.BeautifulSoup(r.text, 'lxml-xml')

    for route in wtdata.find_all('predictions'):
        route_name = route.get('routeTag')
        stop_tag = route.get('stopTag')    
        for prediction in route.find_all('prediction'):
            vehicle_id = prediction.get('seconds')
            seconds = int(prediction.get('seconds'))
            #print(route_name, seconds, stop_tag)
            stops[route_name][stop_tag].next_buses.append(seconds)





In [5]:

origins = [[-84.404, 33.778], [-84.389, 33.776], [-84.389, 33.776], [-84.389, 33.776]]
destinations = [[-84.404, 33.778], [-84.389, 33.776], [-84.389, 33.776], [-84.389, 33.776]]


def request_matrix(origins, destinations):
    
    '''
        #MAPBOX API https://www.mapbox.com/api-documentation/?language=Python#retrieve-a-matrix
        Funtion to obtain the walking time between n origins and m desitnations
        It returns a n x m matrix with the time in seconds (int) and None if is not possible
        to go from i to j
    '''
    ##################################
    # TEMPORARY to avoid API limits #
    #################################
    m = []
    for origin in origins:
        l = []
        for destination in destinations:
            l.append(app_walking_time(origin, destination))
        m.append(l.copy())
    return m

    ################################
            
        
    
    key = 'pk.eyJ1IjoiZ29uemFsb3ZhcmdhcyIsImEiOiJjamc0NHYwenMyN2xtMnFwYzRodzM2aHozIn0.pa9XDqDdS34pcLScaemfHA'
    #coordinates_string ='-122.42,37.78;-122.45,37.91;-122.48,37.73'
    coordinates_string = ';'.join(map(lambda x: ','.join(map(str, x)), origins + destinations))
    ol = len(origins)
    dl = len(destinations)
    index_origin = ';'.join(map(str, range(ol)))
    index_destination = ';'.join(map(str, range(ol, ol+dl)))
    url = 'https://api.mapbox.com/directions-matrix/v1/mapbox/walking/{}?sources={}&destinations={}&access_token={}'.format(coordinates_string, index_origin, index_destination, key)
    r = requests.get(url)    
    return r.json()['durations']
    
request_matrix(origins, destinations)
    

    

[[0.0, 1250.9736652348445, 1250.9736652348445, 1250.9736652348445],
 [1250.9736652348445, 0.0, 0.0, 0.0],
 [1250.9736652348445, 0.0, 0.0, 0.0],
 [1250.9736652348445, 0.0, 0.0, 0.0]]

In [6]:
INFINITE = 99999999

waitp = 1
walkp = 1
ridep = 1


origin = [-84.404, 33.778]
destination = [-84.389, 33.776]

def get_time_matrix(origin, destination, route, stops):
    '''
    Function to obtain the times between origin, destination and stops.
    return:
    -statr = 1 x (S+1) matrix with seconds (int) from origin.
    statr[0][0] is the walking time from origin to destination
    statr[0][s+1] is the walking time from origin to start stop s
    -endr = E x (1) matrix with seconds (int) from end stop to destination.
    endr[0][s] is the walking time from endstop s to destination.
    
    start_stops: list with the stops sorted by eucladian distance
    from origin in ascending order, with limit 24.
    
    end_stops: list with the stops sorted by eucladian distance
    from destination in ascending order, with limit 25.
    '''
    start_stops = list(stops[route].values()).copy()
    start_stops.sort(key=lambda x: distancepp(x.p, origin))
    start_stops = start_stops[:4]
    end_stops = list(stops[route].values()).copy()
    end_stops.sort(key=lambda x: distancepp(x.p, destination))
    end_stops = end_stops[:4]
    startr = request_matrix([origin], [destination] + list(map(lambda x: x.p, start_stops)))
    endr = request_matrix(list(map(lambda x: x.p, end_stops)), [destination])    
    return startr, endr, start_stops, end_stops



startr, endr, start_stops, end_stops = get_time_matrix(origin, destination, 'night', stops)
walking_time = startr[0][0]
print('walking time: ', walking_time, 'seconds', walking_time//60, 'minutes')



    
    




walking time:  1250.9736652348445 seconds 20.0 minutes


### Unefficient  Dijktra

In [7]:
startime = time.time()


def init_nodes(start_stops, end_stops):
    '''
    function that initializes the nodes in a dict.
    parameters:
    start_stops and end_stops from get_time_matrix
    
    return dict(key, value)
    key: (i, j) tuple
    cases:
        i == 0 -> if j == 0: origin;  if j == 1: destination
        i == 1 -> start_stop[j]: aux node to start waiting
        i == 2 -> start_stop[j]:  node to take the  bus
        i == 3 -> end_stops[j]: node where the bus arrives
        typical way:
        (0, 0) -> (1, x), -> (2, x) -> (3, y) -> (0, 1)
        
    values: [cost_to(float), predecessor(node: tuple), visited(bool)]
    '''
    nodes = {}
    # origin
    nodes[(0,0)] = [0, None, False] # [cost_to(float), predecessor(node: tuple), visited(bool)]
    # destination
    nodes[(0,1)] = [INFINITE, None, False]
    # start stops and aux start stop (for waiting time)
    for k in range(len(start_stops)):
        nodes[(1, k)] = [INFINITE, None, False]
        nodes[(2, k)] = [INFINITE, None, False]

    #end stops
    for k in range(len(end_stops)):
        nodes[(3, k)] = [INFINITE, None, False]
    return nodes


nodes = init_nodes(start_stops, end_stops)

def init_arcs(start_stops, end_stops, startr, endr):
    
    '''
    function that initializes the all the arcs in a dict.
    parameters:
    start_stops, end_stops, startr, endr from get_time_matrix
    
    return arcs in a dict (key, list):
    
    key: same from init_nodes
    value: list of arcs that starts at key node.
    Each arc is [destination_node, cost]
    '''
    
    arcs = defaultdict(list)
    for i in range(len(start_stops)):
        #walking to stop
        cost = startr[0][i+1] #because startr[0][0] is walking from origin to destination
        arcs[(0,0)].append((((1, i), cost * walkp)))
        #waiting in stop
        costw = stop_waiting_time(start_stops[i], cost)
        arcs[(1, i)].append(((2, i), costw * waitp))
        #travel time
        for j in range(len(end_stops)):
            cost = travel_time(start_stops[i], end_stops[j])
            arcs[(2, i)].append(((3, j), cost * ridep))
    for j in range(len(end_stops)):
        cost = endr[j][0]
        arcs[3, j].append(((0,1), cost * walkp))    
    return arcs

arcs = init_arcs(start_stops, end_stops, startr, endr)
    
    
#####


def compute_dijktra(nodes, arcs):
    
    '''
    Computes dijktra algorithm (not perfectly) to obtain the lowest cost to all nodes from origin
    and knowing the predecessor of each node in the optimal path.
    '''
     
    unvisited_nodes = list(nodes.keys()).copy()
    unvisited_nodes = [node_key for node_key in unvisited_nodes if nodes[node_key][2] == False]
    '''it can be improved using a heap for the unvisited nodes'''
    while len(unvisited_nodes) > 0:
        actual_node = min(unvisited_nodes, key=lambda x: nodes[x][0])
        cost_to_actual_node = nodes[actual_node][0]   
        for arc in arcs[actual_node]:
            destination_node = arc[0]
            arc_cost = arc[1]
            if destination_node in unvisited_nodes:
                if cost_to_actual_node + arc_cost < nodes[destination_node][0]:
                    nodes[destination_node][0] = cost_to_actual_node + arc_cost
                    nodes[destination_node][1] = actual_node
        nodes[actual_node][2] = True
        unvisited_nodes = [node_key for node_key in unvisited_nodes if not nodes[node_key][2]]
    return nodes # it is not necessary because it changes nodes by reference
        
compute_dijktra(nodes, arcs)


def print_results_from_nodes(nodes, start_stops, end_stops):
    print('walking time: ', walking_time)
    print('bus time:', nodes[(0, 1)][0], 'seconds')
    actual_node = (0,1)
    while actual_node is not None:    
        if actual_node == (0, 1):
            print('destination at: {}'.format(nodes[actual_node][0]))
        elif actual_node == (0, 0):
            print('origin at: {}'.format(nodes[actual_node][0]))
        elif actual_node[0] == 1:
            k = actual_node[1]
            print('start waiting {} bus in {} at {}'.format(start_stops[k].route, start_stops[k].name, nodes[actual_node][0]))
        elif actual_node[0] == 2:
            k = actual_node[1]
            print('take the {} bus in {} at {}'.format(start_stops[k].route, start_stops[k].name, nodes[actual_node][0]))
        elif actual_node[0] == 3:
            k = actual_node[1]
            print('arrive to {} at {}'.format(end_stops[k].name, nodes[actual_node][0]))
        actual_node = nodes[actual_node][1]
    


print_results_from_nodes(nodes, start_stops, end_stops)
    
final_result = {
    'walking_time': walking_time,
    'bus_time': nodes[(0, 1)][0]
}
        
    


walking time:  1250.9736652348445
bus time: 10000616.04279537 seconds
destination at: 10000616.04279537
arrive to Techwood Dr & 5th Street at 10000355.467295047
take the night bus in Fitten Hall - Arrival at 10000018.593571302
start waiting night bus in Fitten Hall - Arrival at 19.5935713019161
origin at: 0


In [9]:

stops = init_stops()
add_next_buses_time(stops)

#for route in ['night']:


for route in ['red', 'green', 'blue']:

    
    print("-------ROUTE: ", route)

    origin = [-84.404, 33.778]
    destination = [-84.389, 33.776]

    startr, endr, start_stops, end_stops = get_time_matrix(origin, destination, route, stops)
    nodes = init_nodes(start_stops, end_stops)
    arcs = init_arcs(start_stops, end_stops, startr, endr)
    compute_dijktra(nodes, arcs)
    print_results_from_nodes(nodes, start_stops, end_stops)





-------ROUTE:  red
walking time:  1250.9736652348445
bus time: 10000626.830986926 seconds
destination at: 10000626.830986926
arrive to Techwood Dr & 5th St at 10000361.668939264
take the red bus in Fitten Hall at 10000026.547754714
start waiting red bus in Fitten Hall at 27.54775471402246
origin at: 0
-------ROUTE:  green
walking time:  1250.9736652348445
bus time: 1191.8644516634504 seconds
destination at: 1191.8644516634504
arrive to Transit HUB at 489.54020568430894
take the green bus in Ferst Dr & Hemphill Ave at 298.0
start waiting green bus in Ferst Dr & Hemphill Ave at 180.07770611996918
origin at: 0
-------ROUTE:  blue
walking time:  1250.9736652348445
bus time: 962.9751350356711 seconds
destination at: 962.9751350356711
arrive to Techwood Dr & 5th Street at 702.3258891392102
take the blue bus in 8th St & West Village at 345.0
start waiting blue bus in 8th St & West Village at 145.88450680464473
origin at: 0


In [ ]:
{
    'walking_time': 3333,
    'busing_time': 2222,
    'stages': [
        {
            'lon': lon,
            'lat': lat,
            'arrival_time': '2/13/18 6:32 am',
            'type': 'walking',
            'description': 'walk to STOP 23 red'
        },

    ]
    
    
    
}

